# Create Call Centre Dialog Dataset (CCD) Unlabeled

# Imports and Paths

In [1]:
# import time
import itertools
import numpy as np
import pandas as pd
import random
import re
# from tqdm import tqdm
from transformers import pipeline

from datasets import concatenate_datasets, Dataset, DatasetDict, load_dataset

from pathlib import Path
import sys
cwd = Path.cwd()
common_code_path = cwd.parent.parent / "plp_common_code"

if str(common_code_path) not in sys.path:
    sys.path.append(str(common_code_path))

# from HF_Dataset_Helper import (hf_display_labels, hf_get_labels, hf_dataset_dict_describe, hf_dataset_describe, hf_display_random_dataset_rows, hf_display_dataset_row_idx)

from HF_Dataset_Helper import (hf_dataset_dict_describe, hf_dataset_describe, hf_dataset_display_random_rows, hf_dataset_display_row,
hf_dataset_get_labels, hf_dataset_display_labels)

from regex_helper import (split_paragraph, remove_punctuation_elements, filter_interest_items, 
remove_short_elements, lowercase_list_elements)

# 1.0 Banking77 Dataset
Dataset composed of online banking queries annotated with their corresponding intents.<br>

BANKING77 dataset provides a very fine-grained set of intents in a banking domain. It comprises 13,083 customer service queries labeled with 77 intents. It focuses on fine-grained single-domain intent detection.
- https://huggingface.co/datasets/PolyAI/banking77

In [2]:
ds = load_dataset("PolyAI/banking77", trust_remote_code=True)
hf_dataset_dict_describe(ds)

type                : <class 'datasets.dataset_dict.DatasetDict'>

shape              : {'train': (10003, 2), 'test': (3080, 2)}

keys               : dict_keys(['train', 'test'])

column_names        : {'train': ['text', 'label'], 'test': ['text', 'label']}

Labels
train set / "label" is type ClassLabel -->
['activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawal_not_recognised', 'change_pin', 'compromised_card', 'contactless_not_working', 'country_support', 'declined_card_payment', 'declined_cash_withdrawal', 'declined_transfer', 'dire

In [3]:
ds_train = ds['train']
ds_test = ds['test']

In [4]:
hf_dataset_display_labels(ds_test)

0  --> activate_my_card
1  --> age_limit
2  --> apple_pay_or_google_pay
3  --> atm_support
4  --> automatic_top_up
5  --> balance_not_updated_after_bank_transfer
6  --> balance_not_updated_after_cheque_or_cash_deposit
7  --> beneficiary_not_allowed
8  --> cancel_transfer
9  --> card_about_to_expire
10 --> card_acceptance
11 --> card_arrival
12 --> card_delivery_estimate
13 --> card_linking
14 --> card_not_working
15 --> card_payment_fee_charged
16 --> card_payment_not_recognised
17 --> card_payment_wrong_exchange_rate
18 --> card_swallowed
19 --> cash_withdrawal_charge
20 --> cash_withdrawal_not_recognised
21 --> change_pin
22 --> compromised_card
23 --> contactless_not_working
24 --> country_support
25 --> declined_card_payment
26 --> declined_cash_withdrawal
27 --> declined_transfer
28 --> direct_debit_payment_not_recognised
29 --> disposable_card_limits
30 --> edit_personal_details
31 --> exchange_charge
32 --> exchange_rate
33 --> exchange_via_app
34 --> extra_charge_on_statement
3

In [5]:
hf_dataset_display_random_rows(3, ds['train'])

row 1:              
Text: Is my top up not working?
Label idx: 47 --> pending_top_up

row 2:              
Text: I would like to get both a Visa and MasterCard?
Label idx: 73 --> visa_or_mastercard

row 3:              
Text: I received a different amount of cash than what I entered.
Label idx: 75 --> wrong_amount_of_cash_received



In [6]:
hf_dataset_display_random_rows(3, ds['test'])

row 1:              
Text: Do all ATMs take this card?
Label idx: 3 --> atm_support

row 2:              
Text: How do I enable contactless pay?
Label idx: 23 --> contactless_not_working

row 3:              
Text: How do I re-add a card to the app?
Label idx: 13 --> card_linking



In [7]:
ds_combined = concatenate_datasets([ds_train, ds_test])
ds_combined_text = ds_combined['text']
len(ds_combined_text)

13083

In [8]:
print("'ds_combined_text' has text only, no labels")
print(f"type: {type(ds_combined_text)}")
print(f"total rows: {len(ds_combined_text):,}")
print("=" * 50, '\n')
for i in range(5):
    print(ds_combined_text[i])

'ds_combined_text' has text only, no labels
type: <class 'list'>
total rows: 13,083

I am still waiting on my card?
What can I do if my card still hasn't arrived after 2 weeks?
I have been waiting over a week. Is the card still coming?
Can I track my card while it is in the process of delivery?
How do I know if I will get my card, or if it is lost?


# 2.0 Minds14 Dataset
MINDS-14 is training and evaluation resource for intent detection task with spoken data. It covers 14 intents extracted from a commercial system in the e-banking domain, associated with spoken examples in 14 diverse language varieties.
- https://huggingface.co/datasets/PolyAI/minds14

In [9]:
ds = load_dataset("PolyAI/minds14", name="en-US", trust_remote_code=True)
type(ds)

datasets.dataset_dict.DatasetDict

In [10]:
hf_dataset_dict_describe(ds)

type                : <class 'datasets.dataset_dict.DatasetDict'>

shape              : {'train': (563, 6)}

keys               : dict_keys(['train'])

column_names        : {'train': ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id']}

Labels
train set / "intent_class" is type ClassLabel -->
['abroad', 'address', 'app_error', 'atm_limit', 'balance', 'business_loan', 'card_issues', 'cash_deposit', 'direct_debit', 'freeze', 'high_value_payment', 'joint_account', 'latest_transactions', 'pay_bill']

train set / "lang_id" is type ClassLabel -->
['cs-CZ', 'de-DE', 'en-AU', 'en-GB', 'en-US', 'es-ES', 'fr-FR', 'it-IT', 'ko-KR', 'nl-NL', 'pl-PL', 'pt-PT', 'ru-RU', 'zh-CN']



In [11]:
ds_train = ds['train']

In [12]:
hf_dataset_display_labels(ds_train, lbl_col="intent_class")

0  --> abroad
1  --> address
2  --> app_error
3  --> atm_limit
4  --> balance
5  --> business_loan
6  --> card_issues
7  --> cash_deposit
8  --> direct_debit
9  --> freeze
10 --> high_value_payment
11 --> joint_account
12 --> latest_transactions
13 --> pay_bill


In [13]:
hf_dataset_display_random_rows(3, ds['train'], lbl_col="intent_class", text_col="english_transcription")

row 1:              
Text: I want to stop all transactions this card
Label idx: 9 --> freeze

row 2:              
Text: what is the most money I can withdraw at one time from an ATM
Label idx: 3 --> atm_limit

row 3:              
Text: I want to stop all transactions this card
Label idx: 9 --> freeze



In [14]:
# combine bank77 and minds14
type(ds['train']['english_transcription']), len(ds['train']['english_transcription'])

(list, 563)

In [15]:
print(len(ds_combined_text))
ds_combined_text = list(itertools.chain(ds_combined_text, ds['train']['english_transcription']))
print(len(ds_combined_text))

13083
13646


# 3.0 Schema-Guided Dialog (SGD) DSTC8
- https://huggingface.co/datasets/google-research-datasets/schema_guided_dstc8

In [16]:
# ds = load_dataset("schema_guided_dstc8", trust_remote_code=True)
ds = load_dataset("google-research-datasets/schema_guided_dstc8", trust_remote_code=True)
# google-research-datasets/schema_guided_dstc8
print(type(ds)), print(ds.shape)

<class 'datasets.dataset_dict.DatasetDict'>
{'train': (16142, 3), 'validation': (2482, 3), 'test': (4201, 3)}


(None, None)

In [17]:
desired_domain = "Banks_1"

def is_desired_domain(services: list):
    for svc in services:
        if svc == desired_domain:
            return True
        else:
            return False

bank_data = ds.filter(lambda x: is_desired_domain(x['services']))
bank_data

DatasetDict({
    train: Dataset({
        features: ['dialogue_id', 'services', 'turns'],
        num_rows: 265
    })
    validation: Dataset({
        features: ['dialogue_id', 'services', 'turns'],
        num_rows: 0
    })
    test: Dataset({
        features: ['dialogue_id', 'services', 'turns'],
        num_rows: 0
    })
})

In [18]:
ds = bank_data['train']
ds

Dataset({
    features: ['dialogue_id', 'services', 'turns'],
    num_rows: 265
})

In [19]:
# let's take a look a one of the Banking multi-turn dialog from the SGD dataset
# Random selection

max_idx = len(ds)-1

speaker_type = {0: "user", 1: "agent"}

counter = 0
max_counter = 1
while counter < max_counter:
    idx = random.randint(0, max_idx)
    dialogue = ds[idx]
    print(f"Dialogue ID: {dialogue['dialogue_id']}")
    print(f"Services: {', '.join(dialogue['services'])}")
    print("\nConversation:")
    print("=" * 50)
    turns = dialogue["turns"]
    for s, u, f in zip(turns["speaker"], turns["utterance"], turns["frames"]):
        print(f"{speaker_type[s]}:\n{u}\n")
    
    counter += 1    

Dialogue ID: 32_00026
Services: Banks_1

Conversation:
user:
Before I pay my bills for the month I would like to check my balance.

agent:
Okay, what account would you like me to look into for you?

user:
Can you please give me the balance of my checking account?

agent:
The current balance of your checking account is $11,440.25.

user:
Thanks, now can you give me the balance of my savings account?

agent:
The current balance of you savings account is $2,660.69.

user:
Okay, that's about what I expected.

agent:
Did you need me to start a transaction for you?

user:
Yes, I do need to make a transfer.

agent:
Okay,I just need a few details first. Who do you need me to make the transfer to and for how much?

user:
I would like you to send 990 bucks to Srinivas please.

agent:
Okay, before I proceed I just need you to verify that you would like to send $990 from your savings account to the checking account belonging to Srinivas.

user:
Yes, all the details you have are correct.

agent:
I 

In [20]:
# take a look at a specific multi-turn dialog
for multi_turn in ds["turns"][0:1]:
    for s, u, _ in zip(multi_turn["speaker"], multi_turn["utterance"], multi_turn["frames"]):    
        print(f"{speaker_type[s]}:\n{u}\n")

user:
What's my balance?

agent:
In checking or savings?

user:
In checking.

agent:
Your checking account has $5,118.77.

user:
Ok, I want to transfer some money.

agent:
To whom?

user:
To Amir.

agent:
And what amount would you like to transfer?

user:
Send $1,630.

agent:
Please confirm: Transfer $1,630 from your checking account to Amir's checking account.

user:
Confirmed.

agent:
Your transfer is complete.

user:
Thanks, what's my balance?

agent:
Your checking account has $3,488.77.

user:
Ok, bye.

agent:
Have a nice day.



In [21]:
# now let's take out the text into a temp list, we only want the customer utterance
ds_sgd_text = []
for multi_turn in ds["turns"]:
    for s, u, _ in zip(multi_turn["speaker"], multi_turn["utterance"], multi_turn["frames"]):   
        if speaker_type[s] == 'user':
            ds_sgd_text.append(u)
        # ds_sgd_text.append(u)

print(type(ds_sgd_text))
print(len(ds_sgd_text))
print(ds_sgd_text[0])
print(ds_sgd_text[1])
print(ds_sgd_text[2])

<class 'list'>
2139
What's my balance?
In checking.
Ok, I want to transfer some money.


In [22]:
# add the SGD customer dialog text into the combined text
print(len(ds_combined_text))
ds_combined_text = list(itertools.chain(ds_combined_text, ds_sgd_text))
print(len(ds_combined_text))
print(type(ds_combined_text))

13646
15785
<class 'list'>


# 4.0 Prepare the text dialog for Active Learning
- Keep only customer dialog paragraph (should have been taken care above)
- break up the dialog paragraph into parts of sentences
- keep only components that have items of interest
    - even if semantic meaning is incorrect, that's ok, the Active Learning and classifier will take care of it
    - if no item of interest, drop the sentence <u>because the "feedback" fall back intent is only fallback for the action, not the product/service.</u>
    - at least 5 words coz portions like "this is good", "hi' are not useful for us.

In [23]:
# Define the items of interest
# todo: load from excel or database (on premise or cloud)
items_of_interest = [
    "current account", "fixed deposit", "foreign exchange", "forex account", "savings account", "mobile app", 
    "credit card", "debit card",
    "account", "app", "atm", "card", "credit", "currency", "current", "debit", "deposit", "exchange", "fixed", 
    "forex", "loan",  "mobile", "rate", "savings", "website"
]  
# items_of_interest = ["atm", "loan",  "mobile app", "savings account"]

In [24]:
print(len(ds_combined_text))
print(type(ds_combined_text))

15785
<class 'list'>


In [25]:
data_1 = []  # data we want to keep
dropped_data = []
for s in ds_combined_text:
    result = split_paragraph(s)
    # print(result)
    
    result = remove_punctuation_elements(result)
    # print(result)

    result_before_filtering = result
    
    # Apply the filter to keep elements that have items of interest in it.
    result = filter_interest_items(result, items_of_interest)
    # print(result)
    
    # remove if list element is < 5 words as it is unlikely to be meaningful    
    result = remove_short_elements(result, min_word_threshold=2)
    # print(result)

    missing_elements = list(set(result_before_filtering) - set(result))
    if missing_elements is not None and len(missing_elements) > 0:
        dropped_data.append(missing_elements)

    if len(result) > 0:  # we don't want empty list 
        data_1.append(s)


dftemp = pd.DataFrame({'dropped_text': dropped_data})
dftemp.to_csv('dropped_data.csv', index=False)
del dropped_data
del dftemp

print(len(data_1))
for i in range(5):
    print(data_1[i])

7706
I am still waiting on my card?
What can I do if my card still hasn't arrived after 2 weeks?
I have been waiting over a week. Is the card still coming?
Can I track my card while it is in the process of delivery?
How do I know if I will get my card, or if it is lost?


In [26]:
# lowercase the strings

data_1 = lowercase_list_elements(data_1)
len(data_1)

7706

# 5.0 Save Call Centre Dialog Dataset (CCD)
- As I do not have the actual dataset from the bank, the CCD represents the "actual" customer dialog that a bank would give me to perform analysis on what areas they can improve on.
- For this purpose, it would be inappropriate to correct any class imbalance.
- For example, if analysis shows that 99% of the dialog is of negative sentiment polarity and only 1% is positive, it means that bank really needs to do something to improve it's products and services.

##### The dialog identifier and the date can/should be modified to what makes sense for a different deployment 
Below is what is created specifically for this POC.

In [27]:
from datetime import date, datetime
today = date.today()
now = datetime.now()

dt_str = str(today.year).zfill(4) + '-' + str(today.month).zfill(2) + '-' + str(today.day).zfill(2)
time_str = str(now.hour).zfill(2) + str(now.minute).zfill(2)
dt_time_str = dt_str + '_' + time_str
dlg_ids = [dt_time_str + '--' + str(i) for i in range(len(data_1))]

In [28]:
# replace this date with the actual date from the data, for the purpose of this POC, we fix an arbitratury date
date_str = '2024-10-05'

data_0 = {
    'date': [date_str for i in range(len(data_1))],
    'dlg_id': dlg_ids,
    'text': data_1
}
df = pd.DataFrame(data_0)
print(df.shape)
display(df.sample(10))

(7706, 3)


date                 dlg_id  \
6279  2024-10-05  2024-10-07_1017--6279   
4368  2024-10-05  2024-10-07_1017--4368   
310   2024-10-05   2024-10-07_1017--310   
2963  2024-10-05  2024-10-07_1017--2963   
3471  2024-10-05  2024-10-07_1017--3471   
1330  2024-10-05  2024-10-07_1017--1330   
1832  2024-10-05  2024-10-07_1017--1832   
2808  2024-10-05  2024-10-07_1017--2808   
423   2024-10-05   2024-10-07_1017--423   
5846  2024-10-05  2024-10-07_1017--5846   

                                                   text  
6279        can i use this account to receive my salary  
4368  the other bank confirmed a transfer i made but...  
310      where are you getting your exchange rates from  
2963  my refund is not showing up in my account wher...  
3471  can you tell me why my card isnt letting me ma...  
1330  how can i get cash back the atm just gave me t...  
1832  i made a deposit and the balance hasnt been up...  
2808  i need to know when a payment will actually go...  
423   there was a problem with the exchange rate on ...  
5846       how do i transfer money using my credit card

In [29]:
df.to_parquet('CCD_Unlabeled.parquet', engine='pyarrow', index=False)

In [30]:
df.to_csv('CCD_Unlabeled.csv', index=False)

In [31]:
data_0 = [{"text": row} for row in ds_combined_text]
print(type(data_0))
i = 0
j = 3
for row in data_0:
    if i > 3:
        break
    for k, v in row.items():
        print(k, ": ", v)
    i += 1

combined_ds = Dataset.from_list(data_0)
combined_ds.save_to_disk('CCD_Unlabeled')

<class 'list'>
text :  I am still waiting on my card?
text :  What can I do if my card still hasn't arrived after 2 weeks?
text :  I have been waiting over a week. Is the card still coming?
text :  Can I track my card while it is in the process of delivery?


Saving the dataset (0/1 shards):   0%|          | 0/15785 [00:00<?, ? examples/s]